11.4 Filtering and recommending

In [ ]:
def mean_squared_error(predictions, actual_values):
    """
    Tính toán sai số bình phương trung bình giữa các giá trị dự đoán và giá trị thực tế.

    Args:
        predictions: Một danh sách hoặc mảng các giá trị dự đoán.
        actual_values: Một danh sách hoặc mảng các giá trị thực tế.

    Returns:
        Sai số bình phương trung bình.
    """
    if len(predictions) != len(actual_values):
        raise ValueError("Số lượng giá trị dự đoán và giá trị thực tế phải bằng nhau.")

    squared_errors = [(p - a) ** 2 for p, a in zip(predictions, actual_values)]
    return sum(squared_errors) / len(squared_errors)

# Ví dụ sử dụng:
predictions = [3, 4, 5, 2, 1]
actual_values = [3, 4, 4, 1, 2]
mse = mean_squared_error(predictions, actual_values)
print(f"Sai số Bình phương Trung bình: {mse}")

Sai số Bình phương Trung bình: 0.6


In [1]:
def collaborative_filtering_nearest_neighbors(user_ratings, target_user, k=2):
    """
    Dự đoán xếp hạng cho một người dùng mục tiêu bằng cách sử dụng lọc cộng tác dựa trên láng giềng gần nhất.

    Args:
        user_ratings: Một từ điển trong đó các khóa là người dùng và các giá trị là các từ điển
                      chứa xếp hạng các mục (mục: xếp hạng).
        target_user: Người dùng mà bạn muốn dự đoán xếp hạng.
        k: Số lượng láng giềng gần nhất để sử dụng.

    Returns:
        Một từ điển chứa các xếp hạng dự đoán cho người dùng mục tiêu.
    """

    def euclidean_distance(user1_ratings, user2_ratings):
        """Tính khoảng cách Euclid giữa xếp hạng của hai người dùng."""
        common_items = set(user1_ratings.keys()) & set(user2_ratings.keys())
        if not common_items:
            return float('inf')  # Không có mục chung, khoảng cách là vô cùng

        distance = sum([(user1_ratings[item] - user2_ratings[item]) ** 2
                         for item in common_items])
        return distance ** 0.5

    # Tính khoảng cách đến những người dùng khác
    distances = {}
    for user, ratings in user_ratings.items():
        if user != target_user:
            distances[user] = euclidean_distance(user_ratings[target_user], ratings)

    # Lấy k láng giềng gần nhất
    nearest_neighbors = sorted(distances.items(), key=lambda item: item[1])[:k]

    # Dự đoán xếp hạng cho người dùng mục tiêu
    predictions = {}
    for item in set().union(*(user_ratings[user].keys() for user, _ in nearest_neighbors)):
        if item not in user_ratings[target_user]:
            weighted_sum = 0
            total_weight = 0
            for user, distance in nearest_neighbors:
                if item in user_ratings[user]:
                    weighted_sum += user_ratings[user][item] / (distance + 1e-10)  # Thêm một hằng số nhỏ để tránh chia cho 0
                    total_weight += 1 / (distance + 1e-10)
            if total_weight > 0:
                predictions[item] = weighted_sum / total_weight
            else:
                predictions[item] = 0  # Hoặc một xếp hạng mặc định nào đó

    return predictions

# Ví dụ sử dụng
user_ratings = {
    'user1': {'item1': 5, 'item2': 3, 'item3': 4},
    'user2': {'item1': 4, 'item2': 5, 'item4': 2},
    'user3': {'item2': 4, 'item3': 3, 'item4': 5},
    'target_user': {'item1': 3, 'item3': 4}
}

target_user = 'target_user'
predictions = collaborative_filtering_nearest_neighbors(user_ratings, target_user)
print(f"Xếp hạng dự đoán cho {target_user}: {predictions}")

Xếp hạng dự đoán cho target_user: {'item2': 4.5, 'item4': 3.5}


In [ ]:
import numpy as np
from sklearn.cluster import KMeans
from collections import defaultdict

def collaborative_filtering_with_clustering(user_ratings, target_user, num_clusters=3):
    """
    Dự đoán xếp hạng cho một người dùng mục tiêu bằng cách sử dụng lọc cộng tác với phân cụm.

    Args:
        user_ratings: Một từ điển trong đó các khóa là người dùng và các giá trị là các từ điển
                      chứa xếp hạng các mục (mục: xếp hạng).
        target_user: Người dùng mà bạn muốn dự đoán xếp hạng.
        num_clusters: Số lượng cụm để sử dụng cho phân cụm người dùng.

    Returns:
        Một từ điển chứa các xếp hạng dự đoán cho người dùng mục tiêu.
    """

    # 1. Tạo ma trận người dùng-mục
    users = list(user_ratings.keys())
    items = set()
    for user in user_ratings.values():
        items.update(user.keys())
    items = list(items)

    user_index = {user: i for i, user in enumerate(users)}
    item_index = {item: j for j, item in enumerate(items)}

    matrix = np.zeros((len(users), len(items)))
    for user, ratings in user_ratings.items():
        for item, rating in ratings.items():
            matrix[user_index[user], item_index[item]] = rating

    # 2. Thực hiện phân cụm người dùng bằng k-means
    kmeans = KMeans(n_clusters=num_clusters, random_state=0, n_init="auto")  # Đã thêm n_init để xử lý thay đổi trong giá trị mặc định
    user_clusters = kmeans.fit_predict(matrix)

    # 3. Dự đoán xếp hạng cho người dùng mục tiêu
    target_user_index = user_index[target_user]
    target_user_cluster = user_clusters[target_user_index]

    cluster_users = defaultdict(list)
    for i, cluster in enumerate(user_clusters):
        cluster_users[cluster].append(users[i])

    predictions = {}
    for item in items:
        if item not in user_ratings[target_user]:
            ratings_sum = 0
            count = 0
            for user in cluster_users[target_user_cluster]:
                if user != target_user and item in user_ratings[user]:
                    ratings_sum += user_ratings[user][item]
                    count += 1
            if count > 0:
                predictions[item] = ratings_sum / count
            else:
                predictions[item] = 0  # Hoặc một xếp hạng mặc định nào đó

    return predictions

# Ví dụ sử dụng
user_ratings = {
    'user1': {'item1': 5, 'item2': 3, 'item3': 4, 'item4': 1},
    'user2': {'item1': 4, 'item2': 5, 'item4': 2, 'item5': 3},
    'user3': {'item2': 4, 'item3': 3, 'item4': 5, 'item6': 4},
    'user4': {'item1': 2, 'item5': 4, 'item6': 5},
    'target_user': {'item1': 3, 'item3': 4, 'item5': 2}
}

target_user = 'target_user'
predictions = collaborative_filtering_with_clustering(user_ratings, target_user)
print(f"Xếp hạng dự đoán cho {target_user} (với phân cụm): {predictions}")

InvalidParameterError: The 'n_init' parameter of KMeans must be a str among {'auto'} or an int in the range [1, inf). Got 'warn' instead.

11.5 11.5 Peer-to-Peer and Metasearch:

In [ ]:
def metasearch(query, search_engines):
    """
    Thực hiện tìm kiếm meta bằng cách truy vấn nhiều công cụ tìm kiếm và hợp nhất kết quả.

    Args:
        query: Chuỗi truy vấn tìm kiếm.
        search_engines: Một từ điển đại diện cho các công cụ tìm kiếm khác nhau.
                       Các khóa là tên công cụ tìm kiếm và các giá trị là các hàm
                       nhận một truy vấn và trả về một danh sách kết quả.

    Returns:
        Một danh sách hợp nhất các kết quả tìm kiếm từ tất cả các công cụ tìm kiếm.
    """

    results = {}
    for engine_name, search_function in search_engines.items():
        results[engine_name] = search_function(query)

    # Hợp nhất đơn giản: nối các kết quả (bạn có thể muốn xếp hạng/khử trùng phức tạp hơn)
    merged_results = []
    for engine_results in results.values():
        merged_results.extend(engine_results)

    return merged_results

# Các hàm công cụ tìm kiếm giả
def search_engine_google(query):
    return [f"Kết quả Google 1 cho '{query}'", f"Kết quả Google 2 cho '{query}'"]

def search_engine_bing(query):
    return [f"Kết quả Bing 1 cho '{query}'", f"Kết quả Bing 2 cho '{query}'", f"Kết quả Bing 3 cho '{query}'"]

def search_engine_duckduckgo(query):
    return [f"Kết quả DuckDuckGo 1 cho '{query}'"]

# Ví dụ sử dụng
search_engines = {
    "Google": search_engine_google,
    "Bing": search_engine_bing,
    "DuckDuckGo": search_engine_duckduckgo
}

query = "lập trình python"
merged_results = metasearch(query, search_engines)

print(f"Kết quả Tìm kiếm Meta cho '{query}':")
for result in merged_results:
    print(f"- {result}")

Kết quả Tìm kiếm Meta cho 'lập trình python':
- Kết quả Google 1 cho 'lập trình python'
- Kết quả Google 2 cho 'lập trình python'
- Kết quả Bing 1 cho 'lập trình python'
- Kết quả Bing 2 cho 'lập trình python'
- Kết quả Bing 3 cho 'lập trình python'
- Kết quả DuckDuckGo 1 cho 'lập trình python'


In [ ]:
class Node:
    """
    Đại diện cho một nút trong mạng P2P.
    """
    def __init__(self, node_id, resources):
        self.node_id = node_id
        self.resources = resources  # Một danh sách các tài nguyên mà nút này có

    def __repr__(self):
        return f"Nút {self.node_id} với các tài nguyên: {self.resources}"

# Ví dụ sử dụng: Tạo một vài nút
node1 = Node("node1", ["file1.txt", "image2.jpg"])
node2 = Node("node2", ["file2.pdf", "video1.mp4", "file1.txt"])
node3 = Node("node3", ["image1.png", "file3.docx"])

# Một biểu diễn đơn giản của mạng P2P (bạn có thể có một cấu trúc phức tạp hơn)
p2p_network = [node1, node2, node3]

# Ví dụ: Tìm các nút có một tài nguyên cụ thể
def find_nodes_with_resource(network, resource_name):
    """
    Tìm các nút trong mạng có một tài nguyên cụ thể.

    Args:
        network: Một danh sách các đối tượng Node.
        resource_name: Tên của tài nguyên cần tìm kiếm.

    Returns:
        Một danh sách các đối tượng Node có tài nguyên đó.
    """
    nodes_with_resource = [node for node in network if resource_name in node.resources]
    return nodes_with_resource

resource_to_find = "file1.txt"
nodes_found = find_nodes_with_resource(p2p_network, resource_to_find)

print(f"Các nút có tài nguyên '{resource_to_find}':")
for node in nodes_found:
    print(f"- {node}")

Các nút có tài nguyên 'file1.txt':
- Nút node1 với các tài nguyên: ['file1.txt', 'image2.jpg']
- Nút node2 với các tài nguyên: ['file2.pdf', 'video1.mp4', 'file1.txt']
